In [70]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_adesai6" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]
mchar = 'm'

beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

source = "https://sourceforge.net/directory/?q=" + mchar + "&sort=name&page="
rest = "https://sourceforge.net/rest/p/"

gleft = 20

header = {'per_page': '99'}

gitlaburls = []
sourceurls = []
hasgit = False



# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

def projectreturns(url):
    r = requests.get(url)
    if r.status_code == 200:
        return True
    return False

def getsourceforge():
    page = 1
    projectnum = 0
    
    file = open('sourceforgelinks.txt', 'w') 

    while True:
        resp = requests.get(source + str(page))
        text = resp.text
        soup = BeautifulSoup(text, 'html.parser')
        if re.search('No results found.', soup.get_text()):
            return

        for link in soup.find_all(class_="project-icon", href=True):
            hasgit = False       
            if projectnum >= 50:
                file.close()
                return
            name = re.findall('/projects/([A-Za-z0-9\-]*)', link.get('href'))
            name = name[0] if name else None
            if name is not None and name.lower().startswith(mchar):
                resp = requests.get(rest + name)
                if resp.status_code == 200:
                    info = json.loads(resp.text)                  
                    for x in range(0, len(info['tools'])):
                        if info['tools'][x]['name'] == 'git':
                            hasgit = True
                            break
                    info['forge'] = 'sourceforge'
                    fullurl = info['url'] + 'code/'
                    #print(fullurl)
                    if hasgit == True:
                        if projectreturns(fullurl):
                            sourceurls.append(fullurl)
                            file.write(fullurl)
                            file.write('\n')
                            existing = coll.find_one(info)
                            if existing is None:
                                coll.insert_one(info)
                            else:
                                continue
                            projectnum += 1
                        
                        
        page += 1
    return

def getgitlab():

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    project_count = 0

    try:
        r = requests .get(beginurl, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if project_count >= 50:
                    return
                if el['name'].lower().startswith(mchar):
                    if projectreturns(el['http_url_to_repo']):
                        project_count += 1
                        el['forge'] = 'gitlab'
                        coll.insert_one(el)
                        gitlaburls.append(el['http_url_to_repo'])
                        
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if project_count >= 50:
                                return
                            if el['name'].lower().startswith(mchar):
                                if projectreturns(el['http_url_to_repo']):
                                    project_count += 1
                                    el['forge'] = 'gitlab'
                                    coll.insert_one(el)
                                    gitlaburls.append(el['http_url_to_repo'])
                                    
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
x = coll.delete_many({})
print(x.deleted_count, " documents deleted.")

getgitlab()
print("Done with getgitlab")
print(len(gitlaburls))
getsourceforge()
print("Done with get sourceforge") 
print(len(sourceurls))

links = open('gitLinks.txt', 'a')
links.write('\n')
   
for x in gitlaburls:
    links.write(x)
    links.write('\n')

links.close()
    

100  documents deleted.
Done with getgitlab
50
Done with get sourceforge
50
